<a href="https://colab.research.google.com/github/paulomikundu/bot-emo/blob/main/emotional_bot_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random
import numpy as np
from tensorflow import keras

# Download VADER lexicon for sentiment analysis
nltk.download('vader_lexicon')

# Lists for responses
ENCOURAGEMENTS = ["Keep going!", "You’re on the right track!", "Believe in yourself!"]
COMPLIMENTS = ["You're very insightful!", "You always know what to say!", "Your presence makes everything better!"]
ROMANTIC_RESPONSES = ["You light up my world!", "I can't stop thinking about you!", "You're my one and only!"]
ADVICE = ["Don't let small things bother you.", "Take time to relax.", "Find joy in the little things."]
EMOTIONS = ["neutral", "excitement", "anger", "sadness", "joy", "fear"]

class EmotionalChatbot:
    def __init__(self, vocab_size=5000, max_len=20):
        self.sentiment_analyzer = SentimentIntensityAnalyzer()
        self.tokenizer = Tokenizer(num_words=vocab_size)
        self.max_len = max_len
        self.model = self.build_neural_network(vocab_size)
        self.train_model()

    def build_neural_network(self, vocab_size):
        model = keras.Sequential([
            keras.layers.Embedding(input_dim=vocab_size, output_dim=64, input_length=self.max_len),
            keras.layers.LSTM(64, return_sequences=True),
            keras.layers.LSTM(64),
            keras.layers.Dense(32, activation='relu'),
            keras.layers.Dense(len(EMOTIONS), activation='softmax')
        ])
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    def analyze_emotion(self, text):
        sentiment_score = self.sentiment_analyzer.polarity_scores(text)['compound']
        if sentiment_score >= 0.5:
            return "excitement"
        elif sentiment_score <= -0.5:
            return "anger"
        else:
            return "neutral"

    def get_encouragement(self):
        return random.choice(ENCOURAGEMENTS)

    def get_compliment(self):
        return random.choice(COMPLIMENTS)

    def get_romantic_response(self):
        return random.choice(ROMANTIC_RESPONSES)

    def provide_advice(self):
        return random.choice(ADVICE)

    def preprocess_input_data(self, message):
        sequence = self.tokenizer.texts_to_sequences([message])
        padded_sequence = pad_sequences(sequence, maxlen=self.max_len, padding='post')
        return padded_sequence

    def train_model(self):
        # Load the dataset
        try:
            df = pd.read_csv('/content/test.txt', header=None, names=['text', 'label'])
        except Exception as e:
            print(f"Error reading the dataset: {e}")
            return

        # Clean and preprocess data
        df.dropna(inplace=True)  # Drop any rows with NaN values
        df['text'] = df['text'].astype(str).str.strip()  # Ensure text is a string
        df['label'] = df['label'].astype(str).str.strip()  # Ensure label is a string
        df = df[df['label'] != '']  # Keep only rows with non-empty labels

        training_sentences = df['text'].values
        training_labels = df['label'].values

        # Ensure only valid labels are used
        label_mapping = {emotion: idx for idx, emotion in enumerate(EMOTIONS)}
        y_train = np.array([label_mapping[label] for label in training_labels if label in label_mapping])

        if len(y_train) == 0:
            print("No valid labels found in the training data.")
            return

        self.tokenizer.fit_on_texts(training_sentences)
        X_train = pad_sequences(self.tokenizer.texts_to_sequences(training_sentences), maxlen=self.max_len, padding='post')
        y_train_categorical = keras.utils.to_categorical(y_train, num_classes=len(EMOTIONS))

        self.model.fit(X_train, y_train_categorical, epochs=100, verbose=1)

    def generate_response_with_neural_network(self, input_data):
        output = self.model.predict(input_data)
        emotion_idx = np.argmax(output)
        return EMOTIONS[emotion_idx]

    def respond_to_message(self, message):
        emotion = self.analyze_emotion(message)
        if emotion == "excitement":
            return f"Wow, that's exciting! {self.get_encouragement()} {self.get_romantic_response()}"
        elif emotion == "anger":
            return "I sense some frustration. Let's talk about it."
        else:
            preprocessed_input = self.preprocess_input_data(message)
            neural_network_generated_response = self.generate_response_with_neural_network(preprocessed_input)
            return (f"Your words seem {neural_network_generated_response}. "
                    f"{self.get_encouragement()} {self.get_compliment()} {self.get_romantic_response()} "
                    f"Here's a piece of advice for you: {self.provide_advice()}")

# User input interaction
if __name__ == "__main__":
    bot = EmotionalChatbot()
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            break
        response = bot.respond_to_message(user_input)
        print("Bot:", response)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


No valid labels found in the training data.
Bot: Wow, that's exciting! Keep going! I can't stop thinking about you!
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 868ms/step
Bot: Your words seem anger. You’re on the right track! You always know what to say! I can't stop thinking about you! Here's a piece of advice for you: Take time to relax.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Bot: Your words seem anger. Keep going! You're very insightful! You're my one and only! Here's a piece of advice for you: Find joy in the little things.
Bot: Wow, that's exciting! You’re on the right track! You light up my world!
